# RNN/LSTM

## RNN Process Sequence
![RNNProcessSeq](./resource/lec10/RNNProcessSeq.png)

one to many -> Image Captioning  
many to one -> Sentiment Classification  
many to many -> Machine Translation  

## RNN
![RNN](resource/lec10/RNN.png)

RNN 내부에는 hidden state를 가지고 있다. hidden state는  RNN이 새로운 입력 데이터를 받을 때마다 업데이트 된다. 그리고 y로 출력값을 내보낸다.

## RNN Structure

![RNNStructure](resource/lec10/RNNStructure.png)

#### RNN의 구조  
Ht는 ht-1 즉 전의 state와 현재 들어오는 xt에 영향을 받아 출력되게 된다. 이 ht는 다음 ht에 영향을 미치게 되어 과거의 데이터가 미래에 영향을 줄 수 있는 구조를 가진 RNN을 만들 수 있다.

### RNN Graph
![RNNComputeGrp](resource/lec10/RNNComputeGrp.png)

파라미터 w는 매 step마다 동일한 w가 쓰이게된다. function도 마찬가지로 같은 function을 사용한다

### RNN Computation Loss
![RNNComputLoss](resource/lec10/RNNComputLoss.png)

각 step마다 y가 있으면 y_t에 대한 loss를 개별적으로 계산하게 된다. 그리고 RNN의 최정 loss는 각 step의 loss들의 합이다.

## Many to One  + One to Many
마지막 hidden state에 의해 결정되는 y (final hidden state summarize all of the context from the entire sequence)  


Use fixed sized input to initialize hidden state of the model, and the recurrent network will tick for each cell in the output. As you produce your variably sized output, you’ll unroll the graph for each element in the output. 

![MtoOnetoM](resource/lec10/MtoOnetoM.png)

## Sequence to Sequence Model
Encoder (many to one): receive the variably sized input, and then summarize that entire sentence using the final hidden state of the encoder network  


Decoder (one to many): input that single vector, summarizing the input sentence and now produce this variably sized output which might be your sentence in another language. In this variably sized output, we might make some predictions at every time step. Train this entire thing by unrolling this computational graph, summing the losses at the output sequence and performing back propagation.


### Example: Character-level Language Model Sampling (hello)

![SeqToSeqEx](resource/lec10/SeqToSeqEx.png)

- h 만 넣어서 ello라는 output sequence를 예측하는데 사용  
- softmax를 이용해 정답 값과 score의 loss 비교
- back propagation을 통해 학습해 나간다.
- 계속 반복해 나간다.    

*Why sampling instead of choosing highest score?*  
: lets you get diversity from your models


## Truncatedd BackPropagation through time

긴 input sequence를 학습시킨다면 w에다가 너무 많고 다양한 gradient를 누적하게 되어 아주 느리고 성능이 안 좋아지는 현상이 나타날 수 있습니다.  

![RNNBackpropagate](resource/lec10/RNNBackpropagate.png)

→ Compute a loss only over subsequence of the data and then back propagate through this sub sequence. When we make next batch, copy those hidden states forward in time, so the forward pass will be exactly the same. But when we compute gradient step, only backpropagate through this next batch

## Vanilla RNN Gradient Flow

![vanilla_rnn_gradient](resource/lec10/VanillaRnnGradientFlow.png)

### Computing gradient of $h_0$ involves many factors of W (and Repeated tanh)

- W(weight) 가중치 곱은 행렬곱으로 Gradient를 구하는 역전파에서 Transpose 되어야 한다.   
  무수히 많은 W를 Transpose -> much time
- tanh 연산 -> exponential

### Exploding Gradients
- Largest singular value > 1
- 매 State에서 같은 가중치를 곱하므로, 가중치에 값이 1 이상이면, exponential하게 Gradient가 커진다.  


- 해결: Gradient clipping -> Scale gradient..

### Vanishing Gradients
- Largest singular value < 1
- Exploding과 반대로, 가중치 값이 1 이하면, State수가 많을 때 Gradient가 0으로 수렴한다.  


- 이에 대한 해결은, RNN Architecture를 바꾸는 것! -> **LSTM**

## LSTM (Long Short Term Memory)

![LSTM Structure](resource/lec10/LSTMStructure.png)

**Point: Sigmoid와의 곱 = 1, 0 으로 on/off 스위치 기능을 하게 된다**  
- 각 i, f, o, g Gate는 가중치와 행렬곱이 아닌, Elementwise Multiplication을 한다.  

### i , Input Gate
- Whether to write to cell
- 1: 입력 데이터를 반영한다. / 0: 입력 데이터를 반영하지 않는다. 
### f, Forget Gate
- Whether to erase cell
- 1: 과거 데이터를 반영한다. / 0: 과거 데이터를 반영하지 않는다.
### o, Output Gate
- How much to reveal to cell
- 1: 현 스테이트 결과를 다음 스테이트에 내보낸다. / 0: 결과를 다음 스테이트에 내보내지 않는다. 
### g, Gate Gate
- How much to write to cell
- tanh로 입력데이터를 정제한 값

## LSTM Gradient Flow

![LSTMGradientFlow](resource/lec10/LSTMGradientFlow.png)

**Cell Gate 를 통해서 Gradient Flow를 쉬운 연산과 단순한 경로로 Uninterrupted하게 진행시킨다!!**  

- Cell Gate에 가중치가 반영되어 있다.
- Hidden State는 역전파의 시작 State에서 Tanh 에 한번 적용된다. (= tanh gradient 계산은 한번만 수행)